In [1]:
#Packages to edit SQLite in Python, access operating system, use regex
import sqlite3
import os
import re

# Open folder and directory
data = 'C:\\Users\\Wren\\Documents\\THEtool_logfiles' #replace with the correct file path
filenames = []
lowest_t = {} #empty dictionary for removing time specific data
dir_list = os.listdir(data)
for file in dir_list:
    if re.match(r"\d+-\w+-\S+", file):
        filenames.append(file)

# Empty a table in the log
for file in filenames:
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("DELETE FROM props;")
# Clear tabs 
    c.execute("UPDATE act SET json='{\"is_focused\":false}' where k='focus' AND json LIKE '%is_focused\":false%'")
# Find the lowest value of t for each file
    c.execute("SELECT MIN(t) FROM (SELECT MIN(t) AS t FROM act UNION SELECT MIN(t) AS t FROM key UNION SELECT MIN(t) AS t FROM eye);")
    row = c.fetchone()
    if row is not None:
        lowest_t[file] = row[0]
# Subtract the lowest t from all t values in each file
    c.execute("UPDATE act SET t = t - ? WHERE t >= ?;", (lowest_t[file], lowest_t[file]))
    c.execute("UPDATE key SET t = t - ? WHERE t >= ?;", (lowest_t[file], lowest_t[file]))
    c.execute("UPDATE eye SET t = t - ? WHERE t >= ?;", (lowest_t[file], lowest_t[file]))
# Change the name of the log
    current_name = os.path.join(data, file)
    new_name = re.sub(r"(\d+-)\w+(-\S+)", r"\1anon\2", file)
    try:
        conn.commit()  # Commit any pending changes before closing the connection
        conn.close()   # Close the connection to release the file lock
        os.rename(current_name, os.path.join(data, new_name))
    except Exception as e:
        print(f"Skipping {current_name} - unable to rename file ({e})")